In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split

In [2]:
dataset = '../Dataset/Car/car'
#dataset = '../Dataset/Balance-sheet/train.txt'
#dataset = '../Dataset/Hayes-Roth/Hayes-Roth'
#dataset = '../Dataset/Heart-Data/train.txt'
#dataset = '../Dataset/TicTacToe/tic tac toe'
DATASET_SPLIT_RATIO = .1
np.random.seed(2)

In [3]:
dataframe = pd.read_csv(dataset, header=None)
train, test = train_test_split(dataframe, test_size=DATASET_SPLIT_RATIO)

In [4]:
train.head()
print('Number of training samples:', len(train))
print('Number of test samples:', len(test))
print('Labels:', np.unique(train[list(dataframe)[-1]]))

Number of training samples: 1555
Number of test samples: 173
Labels: ['acc' 'good' 'unacc' 'vgood']


In [5]:
def select_feature(data):
    list_of_features = list(data)
    list_of_labels = np.unique(data[list_of_features[-1]], return_counts=True)[0]
    label_count = np.unique(data[list_of_features[-1]], return_counts=True)[1]
    total_number_of_rows = len(data)
    entropy = 0
    for i in range(0, len(list_of_labels)):
        entropy = entropy + (- label_count[i]/total_number_of_rows) * (math.log2(label_count[i]/total_number_of_rows))
        # Total numer of rows in the entire dataset or the number of rows in the current node?
    #print('Entropy:', entropy)

    mx_gain_ratio = -1
    mx_idx = -1

    for i in range(0, len(list_of_features) - 1):
        # calculate information gain of i-th feature using the labels on data[list_of_features[-1]]
        list_of_possible_values = np.unique(data[list_of_features[i]], return_counts=True)[0]
        count_values = np.unique(data[list_of_features[i]], return_counts=True)[1]
        total_values = sum(count_values)
        information = 0
        split_info = 0
        #print(list_of_possible_values)
        #print(count_values)
        for j in range(len(list_of_possible_values)):
            information_label = 0
            for k in range(len(list_of_labels)):
                denominator = len(data[(data[list_of_features[i]] == list_of_possible_values[j]) & (data[list(data)[-1]] == list_of_labels[k])])          
                if denominator == 0:
                    continue
                information_label = information_label + (- denominator / count_values[j]) * math.log2(denominator / count_values[j])
            information = information + information_label * (count_values[j]/total_values)
            split_info = split_info +  -(count_values[j]/total_values) * math.log2(count_values[j]/total_values)
        
        information_gain = entropy - information
        #split_info = 1
        gain_ratio = information_gain / split_info
        #print('Feature', i, ':', information_gain, split_info, gain_ratio)
        if gain_ratio > mx_gain_ratio:
            mx_gain_ratio = gain_ratio
            mx_idx = i

        #print('Feature', i, ':', information)
        #print('Feature', i, ':', information_gain)

    return mx_idx

In [6]:
select_feature(dataframe)

5

In [7]:
def make_decision_tree(node, data):
    global number_of_nodes, node_identity, node_edge
    list_of_features = list(data)
    list_of_labels = np.unique(data[list_of_features[-1]], return_counts=True)[0]
    list_of_labels_count = np.unique(data[list_of_features[-1]], return_counts=True)[1]
    if len(list_of_labels) == 1:
        node_identity[node] = list_of_labels[0]
        return
    if len(list(data)) == 1:
        majority_idx = np.argmax(list_of_labels_count)
        node_identity[node] = list_of_labels[majority_idx] 
        return
    
    mx_idx = select_feature(data)
    #print(node, mx_idx)
    remaining_features = list_of_features[0:mx_idx] + list_of_features[mx_idx+1: ]
    #print('Selected feature:', mx_idx)
    #print('Remaining_features:', remaining_features)
    list_of_possible_values = np.unique(data[list_of_features[mx_idx]])
    node_edge[node] = list_of_features[mx_idx]
    node_edge_dict[node] = {}    
    for label in list_of_possible_values:
        new_data_frame = data[data[list_of_features[mx_idx]] == label][remaining_features]
        if len(new_data_frame) == 0:
            continue
        new_node = number_of_nodes
        node_edge_dict[node][label] = new_node
        node_identity.append(-1)
        node_edge.append(-1)
        node_edge_dict.append(-1)
        number_of_nodes += 1
        make_decision_tree(new_node, new_data_frame)
    return 


In [8]:
number_of_nodes = 1
root_node = 0
node_identity = [-1]
node_edge = [-1]
node_edge_dict = [-1]
make_decision_tree(root_node, train)    

In [9]:
number_of_nodes

382

In [10]:
def predict(data):
    cur_node = 0
    while node_identity[cur_node] == -1:
        try:
            cur_node = node_edge_dict[cur_node][data[node_edge[cur_node]].values[0]]
        except:
            return -1
    return node_identity[cur_node]

In [11]:
predict(test[0:1])

'unacc'

In [12]:
right_predictions = 0
for i in range(len(test)):
    if predict(test[i:i+1]) == test[i:i+1][list(test)[-1]].values[0]:
        right_predictions+=1

In [13]:
print('Accuacy:', str(  round(right_predictions/len(test) * 100, 2)) + '%' )

Accuacy: 93.06%


Car Dataset: 93.06%
Balance Sheet: 26.98%
Hayes-Roth: 50.0%
Heart Data: 89.47%
TicTacToe: 85.42%